In [1]:
## This notebook extracts data from the Japanese Ministry of Health, Labor and Welfare,
## transforms it into its necessary components, where it is then ready for visualization on a
## Tableau Public dashboard.

## Import packages
import pandas as pd
import numpy as np
import datetime as dt
import time
import ssl

## Display settings
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Data Sources, setting up SSL
ssl._create_default_https_context = ssl._create_unverified_context
dataCases = 'http://covid19.mhlw.go.jp/public/opendata/newly_confirmed_cases_daily.csv'
dataSevere = 'http://covid19.mhlw.go.jp/public/opendata/severe_cases_daily.csv'
dataDeaths = 'http://covid19.mhlw.go.jp/public/opendata/deaths_cumulative_daily.csv'

## First day of each COVID wave in Datetime format
dtWaves = [dt.datetime(2020, 6, 1),dt.datetime(2020, 11, 1),dt.datetime(2021, 3, 1),
           dt.datetime(2021, 7, 1),dt.datetime(2022, 1, 1),dt.datetime(2022, 6, 1),
           dt.datetime(2029, 6, 1)]

In [2]:
def pullData():
    ## Create and Join Data Frames
    dfCases = pd.read_csv(dataCases)[['Date','ALL']]
    dfCases.rename(columns = {'ALL':'newCases'}, inplace = True)

    dfSevere = pd.read_csv(dataSevere)[['Date','ALL']]
    dfSevere.rename(columns = {'ALL':'totalSevere'}, inplace = True)

    dfDeaths = pd.read_csv(dataDeaths)[['Date','ALL']]
    dfDeaths.rename(columns = {'ALL':'cumDeaths'}, inplace = True)

    df = dfCases.join(dfSevere.set_index('Date'), on="Date", how='inner')
    df2 = df.join(dfDeaths.set_index('Date'), on="Date", how='inner')

    ## Reset index to 0 after joining
    df2 = df2.reset_index(drop=True)


    ## Inputting each wave into the dataframe, using the dtWaves array above
    def wave(row):
        for i in range(0,len(dtWaves)):
            if dt.datetime.strptime(row['Date'], '%Y/%m/%d') >= dtWaves[i]:
                if dt.datetime.strptime(row['Date'], '%Y/%m/%d') < dtWaves[i + 1]:
                    return "Wave " + str(i+2)
            else:
                return "Wave 1"
    df2['wave'] = df2.apply(wave, axis=1)

    ## Adding column for new deaths per day
    newDeaths = []
    last = 613
    for i in df2["cumDeaths"]:
        newDeaths.append(i - last)
        last = i

    df2.insert(4, "newDeaths", newDeaths, True)

    ## Adding column for average number of weekly cases
    weeklyAvgCases = []
    avg = []

    for i in df2["newCases"]:
        if len(avg) < 7:
            avg.append(i)
            weeklyAvgCases.append(None)
        else:
            del avg[0]
            avg.append(i)
            weeklyAvgCases.append(round(np.average(avg,axis=0),1))

    df2.insert(2, "weeklyAvgCases", weeklyAvgCases, True)

    ## Adding column for average number of weekly deaths
    weeklyAvgDeaths = []
    avg = []

    for i in df2["newDeaths"]:
        if len(avg) < 7:
            avg.append(i)
            weeklyAvgDeaths.append(None)
        else:
            del avg[0]
            avg.append(i)
            weeklyAvgDeaths.append(round(np.average(avg,axis=0),1))

    df2.insert(6, "weeklyAvgDeaths", weeklyAvgDeaths, True)


    ## Adding three columns with a constant value containing all cases, all severe, and all deaths
    totalCases = 0
    totalSevere = 0
    totalDeaths = 0
    for i in df2["newCases"]:
        totalCases += i

    for i in df2["totalSevere"]:
        totalSevere += i

    for i in df2["newDeaths"]:
        totalDeaths += i

    df2['allCases'] = pd.Series([totalCases for x in range(len(df2.index))])
    df2['allSevere'] = pd.Series([totalSevere for x in range(len(df2.index))])
    df2['allDeaths'] = pd.Series([totalDeaths for x in range(len(df2.index))])
    return df2

In [3]:
## Check first rows
df2 = pullData()
df2.head()

,Date,newCases,weeklyAvgCases,totalSevere,cumDeaths,newDeaths,weeklyAvgDeaths,wave,allCases,allSevere,allDeaths
0,2020/5/9,108,NaN,267,613,0,NaN,Wave 1,18654635,439559,39577
1,2020/5/10,66,NaN,249,621,8,NaN,Wave 1,18654635,439559,39577
2,2020/5/11,58,NaN,243,643,22,NaN,Wave 1,18654635,439559,39577
3,2020/5/12,87,NaN,259,668,25,NaN,Wave 1,18654635,439559,39577
4,2020/5/13,55,NaN,245,687,19,NaN,Wave 1,18654635,439559,39577


In [4]:
## Check last rows (including last date in the dataset)
df2.tail()

,Date,newCases,weeklyAvgCases,totalSevere,cumDeaths,newDeaths,weeklyAvgDeaths,wave,allCases,allSevere,allDeaths
841,2022/8/28,157788,192027.4,628,38997,235,285.3,Wave 7,18654635,439559,39577
842,2022/8/29,95898,185581.7,618,39245,248,282.3,Wave 7,18654635,439559,39577
843,2022/8/30,152529,177583.6,591,39556,311,281.4,Wave 7,18654635,439559,39577
844,2022/8/31,169771,167057.9,555,39872,316,284.7,Wave 7,18654635,439559,39577
845,2022/9/1,149868,156908.0,539,40190,318,285.9,Wave 7,18654635,439559,39577


In [5]:
print("Data Pull Started: " + str(dt.datetime.now()))
pullData()
print(df2.shape)
print(df2.tail(1))
df2.to_csv('/Users/davidkatilius/Documents/Tableau_Data/japan-covid-auto-dashboard.csv')
print("Data Pull Complete: " + str(dt.datetime.now()))

Data Pull Started: 2022-09-02 17:25:46.663815
(846, 11)
         Date  newCases  weeklyAvgCases  totalSevere  cumDeaths  newDeaths  \
845  2022/9/1    149868        156908.0          539      40190        318   

     weeklyAvgDeaths    wave  allCases  allSevere  allDeaths  
845            285.9  Wave 7  18654635     439559      39577  
Data Pull Complete: 2022-09-02 17:25:49.152246
